In [10]:
#Importing all libraries
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [12]:
#Headers will make it look like you are using a web browser
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
#We will use the iteration to retrieve and scrape the web pages, reviews, and ratings from each page on Yelp
reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []

df = pd.read_csv('Supercuts Review List.csv')

for i, row in df.iterrows():
    if row['Brand'] == 'Supercuts': 
        url = row['url']
        response = requests.get(url, headers=headers, verify=False).text
        soup = BeautifulSoup(response, "lxml")
    #Looping through 'div' 'review-content' will help find all the review containers we need in each page that have rating and review
        for s in soup.find_all("div", attrs={'class': 'review-content'}):
            re = s.find('p', attrs={'lang': 'en'})
    #Makes all the letters lower in reviews
            review = re.text.lower()
            review_corpus.append([review])
    #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
    #Make sure to to run the cList dict cell or else there will be an error
            expandContractions(review)
    #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
            words = word_tokenize(review)
            words = word_tokenize(review.replace('\n',' '))
            clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
            characters_to_remove = ["''",'``','...']
            clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
            english_stops = set(stopwords.words('english'))
            clean_words = [word for word in clean_words if word not in english_stops]
            wordnet_lemmatizer = WordNetLemmatizer()
            lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
            reviews.append(lemma_list)
    #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
            rating = s.find_all('img', attrs={'class': 'offscreen'})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
            rate = str(rating)
            int_rating = int(rate[11:12])

            if int_rating == 1 or int_rating == 2 or int_rating == 3:
                rating = 'neg'
                ratings.append('neg')
                neg_reviews.append(lemma_list)
            elif int_rating == 4 or int_rating == 5:
                rating = 'pos'
                ratings.append('pos')
                pos_reviews.append(lemma_list)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('haircut_review_corpus_supercuts.csv', index=False, header=False)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

In [13]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [14]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [15]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [16]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [17]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [19]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [21]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.003*"hair" + 0.002*"cut" + 0.002*"great" + 0.002*"go" + 0.002*"get" + '
  '0.001*"good" + 0.001*"time" + 0.001*"haircut" + 0.001*"give" + '
  '0.001*"make"'),
 (1,
  '0.002*"hair" + 0.002*"cut" + 0.002*"great" + 0.001*"go" + 0.001*"haircut" + '
  '0.001*"get" + 0.001*"time" + 0.001*"good" + 0.001*"stylist" + 0.001*"come"'),
 (2,
  '0.003*"hair" + 0.002*"great" + 0.002*"cut" + 0.002*"go" + 0.002*"haircut" + '
  '0.002*"get" + 0.001*"time" + 0.001*"come" + 0.001*"place" + 0.001*"good"'),
 (3,
  '0.002*"hair" + 0.001*"haircut" + 0.001*"cut" + 0.001*"great" + 0.001*"get" '
  '+ 0.001*"time" + 0.001*"go" + 0.001*"good" + 0.001*"stylist" + '
  '0.001*"want"'),
 (4,
  '0.034*"hair" + 0.028*"cut" + 0.018*"great" + 0.018*"get" + 0.017*"go" + '
  '0.017*"haircut" + 0.014*"time" + 0.013*"good" + 0.010*"stylist" + '
  '0.009*"come"'),
 (5,
  '0.002*"hair" + 0.002*"cut" + 0.001*"get" + 0.001*"great" + 0.001*"haircut" '
  '+ 0.001*"go" + 0.001*"good" + 0.001*"stylist" + 0.001*"place" + '
 

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
4     -0.209384 -4.675051e-08       1        1  99.999634
3      0.040515 -5.941493e-05       2        1   0.000045
7      0.036488  1.522623e-05       3        1   0.000043
9      0.036457 -2.308534e-05       4        1   0.000043
5      0.032569  5.210594e-05       5        1   0.000042
1      0.030073  1.157014e-04       6        1   0.000041
8      0.026417  3.806504e-05       7        1   0.000040
2      0.024576 -1.196196e-04       8        1   0.000039
0      0.022521 -2.104499e-05       9        1   0.000038
6     -0.040232  2.113017e-06      10        1   0.000032, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
1284  Default  1016.000000        hair  1016.000000  30.0000  30.0000
722   Default   853.000000         cut   853.000000  29.0000  29.0000
1245  Default   558.000000       great   558.000000  28.0000  28.0000
1218  Default   505.000000          go   505.000000  27.0000  27.0000
1200  Default   542.000000         get   542.000000  26.0000  26.0000
1286  Default   503.000000     haircut   503.000000  25.0000  25.0000
1226  Default   398.000000        good   398.000000  24.0000  24.0000
2981  Default   409.000000        time   409.000000  23.0000  23.0000
2831  Default   312.000000     stylist   312.000000  22.0000  22.0000
581   Default   283.000000        come   283.000000  21.0000  21.0000
98    Default   271.000000      always   271.000000  20.0000  20.0000
2853  Default   280.000000    supercut   280.000000  19.0000  19.0000
3185  Default   255.000000        want   255.000000  18.0000  18.0000
2168  Default   264.000000       place   264.000000  17.0000  17.0000
1752  Default   242.000000        make   242.000000  16.0000  16.0000
1539  Default   249.000000         job   249.000000  15.0000  15.0000
1209  Default   231.000000        give   231.000000  14.0000  14.0000
2890  Default   227.000000        take   227.000000  13.0000  13.0000
238   Default   240.000000        back   240.000000  12.0000  12.0000
3177  Default   209.000000        wait   209.000000  11.0000  11.0000
1703  Default   205.000000        look   205.000000  10.0000  10.0000
3296  Default   185.000000        year   185.000000   9.0000   9.0000
1691  Default   201.000000    location   201.000000   8.0000   8.0000
1150  Default   195.000000    friendly   195.000000   7.0000   7.0000
2504  Default   177.000000       salon   177.000000   6.0000   6.0000
2587  Default   167.000000     service   167.000000   5.0000   5.0000
2944  Default   166.000000       thank   166.000000   4.0000   4.0000
1715  Default   156.000000        love   156.000000   3.0000   3.0000
1595  Default   160.000000        know   160.000000   2.0000   2.0000
2345  Default   151.000000      really   151.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1200  Topic10     0.000043         get   542.385010  -1.3969  -5.4169
1286  Topic10     0.000040     haircut   503.568298  -1.3832  -5.4775
1226  Topic10     0.000036        good   398.580597  -1.2688  -5.5968
2831  Topic10     0.000032     stylist   312.499390  -1.1456  -5.7170
1245  Topic10     0.000040       great   558.866394  -1.4797  -5.4698
2981  Topic10     0.000034        time   409.764984  -1.3501  -5.6505
3185  Topic10     0.000027        want   255.177979  -1.1119  -5.8859
581   Topic10     0.000027        come   283.000153  -1.1918  -5.8624
98    Topic10     0.000026      always   271.103668  -1.1916  -5.9051
2890  Topic10     0.000023        take   227.485367  -1.1286  -6.0175
1752  Topic10     0.000024        make   242.265091  -1.1703  -5.9963
1539  Topic10     0.000023         job   249.390015  -1.2478  -6.0448
2851  Topic10     0.000017       super   125.479630  -0.8743  -6.3581
3177  Topic10     0.000020        wait   

In [23]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.219730  0.000715       1        1  98.803131
5     -0.149185 -0.001175       2        1   1.196710
9      0.072719  0.001049       3        1   0.000028
7      0.068988  0.000914       4        1   0.000026
2      0.068323  0.001099       5        1   0.000026
0      0.054711 -0.000519       6        1   0.000019
8      0.048276 -0.000923       7        1   0.000016
3      0.047930 -0.001362       8        1   0.000016
1      0.029698 -0.001750       9        1   0.000010
4     -0.021732  0.001953      10        1   0.000007, topic_info=     Category          Freq           Term        Total  loglift  logprob
term                                                                     
1284  Default  1.279000e+03   fingerlength  1279.000000  30.0000  30.0000
722   Default  1.084000e+03   conversation  1084.000000  29.0000  29.0000
1200  Default  6.890000e+02       extended   689.000000  28.0000  28.0000
1245  Default  7.220000e+02          fault   722.000000  27.0000  27.0000
1286  Default  6.310000e+02       finished   631.000000  26.0000  26.0000
1218  Default  6.490000e+02        faculty   649.000000  25.0000  25.0000
2981  Default  5.120000e+02          shear   512.000000  24.0000  24.0000
1226  Default  5.060000e+02          false   506.000000  23.0000  23.0000
2853  Default  3.630000e+02        safeway   363.000000  22.0000  22.0000
2831  Default  3.980000e+02           rude   398.000000  21.0000  21.0000
581   Default  3.660000e+02        clarify   366.000000  20.0000  20.0000
2168  Default  3.370000e+02           much   337.000000  19.0000  19.0000
98    Default  3.480000e+02         almost   348.000000  18.0000  18.0000
238   Default  3.030000e+02     availabile   303.000000  17.0000  17.0000
3185  Default  3.240000e+02          stack   324.000000  16.0000  16.0000
1539  Default  3.150000e+02           head   315.000000  15.0000  15.0000
1209  Default  2.960000e+02           eyed   296.000000  14.0000  14.0000
1752  Default  3.070000e+02  intentionally   307.000000  13.0000  13.0000
2890  Default  2.880000e+02          scene   288.000000  12.0000  12.0000
1703  Default  2.610000e+02     indication   261.000000  11.0000  11.0000
3177  Default  2.620000e+02         spritz   262.000000  10.0000  10.0000
1691  Default  2.580000e+02   inconsistent   258.000000   9.0000   9.0000
1150  Default  2.480000e+02      everytime   248.000000   8.0000   8.0000
3296  Default  2.370000e+02      supervise   237.000000   7.0000   7.0000
2504  Default  2.270000e+02            ppl   227.000000   6.0000   6.0000
2944  Default  2.140000e+02        serious   214.000000   5.0000   5.0000
2587  Default  2.110000e+02        publish   211.000000   4.0000   4.0000
1715  Default  1.990000e+02         infact   199.000000   3.0000   3.0000
1595  Default  2.030000e+02      honeycomb   203.000000   2.0000   2.0000
2345  Default  1.950000e+02          owned   195.000000   1.0000   1.0000
...       ...           ...            ...          ...      ...      ...
1286  Topic10  1.307704e-05       finished   631.090332  -1.1825  -5.2933
2981  Topic10  1.164966e-05          shear   512.499634  -1.0899  -5.4089
3475  Topic10  5.133998e-07          train     0.152920   3.9053  -8.5309
1226  Topic10  9.075074e-06          false   506.352936  -1.3276  -5.6586
2853  Topic10  7.879305e-06        safeway   363.031891  -1.1361  -5.7999
1245  Topic10  9.816095e-06          fault   722.252075  -1.6042  -5.5801
1218  Topic10  8.720434e-06        faculty   649.161926  -1.6159  -5.6985
238   Topic10  6.576264e-06     availabile   303.507721  -1.1378  -5.9807
3185  Topic10  6.131384e-06          stack   324.043854  -1.2733  -6.0508
1539  Topic10  6.023062e-06           head   315.383636  -1.2641  -6.0686
1209  Topic10  5.893586e-06           eyed   296.920715  -1.2255  -6.0903
2890  Topic10  5.754592e-06          scene   288.8131